In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'mushroom-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F478%2F974%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240609%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240609T042019Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D869794528a769b25b747fe7bd658137007d590b09fab38472386e058bd90a328ff88fe5e1b498a523ef36cd5e4503d79f2afc044ea8113644fe61f0ad1eeed0cc491ae8fe29d3a84f8765bb130e2834db9914f0034918e9293db7a402afe73c0ec5022daa3a81d2860512576766d2d40cf2a2a90630399f420d0a9cfe925423b8595a2f35a0f4ea808f9c477088792659471a15616b5726a4a43c62cb12d4093885101491f855c5ae2ae868e8bf8dca1a2e3bddca3c1139f6fe2771d32dbfa8a9b505f58925817a46c598b9cb75114ba882cbebea7399e42054161882dd46aeb8bc68db439729443f1ae4ee5a8e30a810063f4fb03e7f029e3935dd337f8c1e6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os

In [ ]:
# loading the mushroom dataset
dataset=pd.read_csv("/kaggle/input/mushroom-classification/mushrooms.csv")

In [ ]:
# lets look at our dataset
print(dataset)

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
#Let's separate input and output to variables X and y
X=dataset.drop(['class'],axis=1)
y=dataset['class']

In [ ]:
# Lets see the shape of X and y
print("Shape of X :{}".format(X.shape))
print("Shape of y: {}".format(y.shape))

In [ ]:
#Since X is object datatype convert it into int by pandas dummies
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
X=enc.fit_transform(X)
print(X)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)
print(y)

In [ ]:
# lets split the X and y for training and testing
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
dt=DecisionTreeClassifier(criterion='gini',max_depth=3,random_state=0)
dt.fit(X_train,y_train)

In [ ]:
plt.figure(figsize=(12,8))
tree.plot_tree(dt.fit(X_train,y_train))

In [ ]:
y_pred=dt.predict(X_test)
y_train_pred=dt.predict(X_train)

In [ ]:
print(y_pred)
print(y_train_pred)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print("Model Accurcay on test data: {}".format(accuracy_score(y_test,y_pred)))
print("Training accuracy:{}".format(accuracy_score(y_train,y_train_pred)))
